# About this Notebook


<h3> Dataset </h3>
In the following notebook, I tried to analyze the attached dataset and explore it more to find the relationship between different features.
Then I applied the EDA, PCA, and feature engineering.

<h4> EDA </h4>

<h3> Model Selection </h3>
Fo modeling the dataset, I considered the Boosted Tree Classifier from the TensorFlow library. 

<h4> Metrics </h4>
The metric I used to measure the model performance is the accuracy of the classifier.

<h4> DFCs </h4>
In this NoteBook, after training the model, I tried to move on to have more detail about the trained model by applying directional feature contributions (DFCs).

</br>

<img src="https://miro.medium.com/max/3798/1*RbBwF9pMlxQm45cVjDch4w.jpeg" alt="Travel" width="500" height="600">

<hr>
I tried to explain each cell in a markdown cell above.


<h5>If you are interested in this problem and detailed analysis, you can copy this Notebook as follows</h5>

<img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1101107%2F8187a9b84c9dde4921900f794c6c6ff9%2FScreenshot%202020-06-28%20at%201.51.53%20AM.png?generation=1593289404499991&alt=media" alt="Copyandedit" width="300" height="300" class="center">

# Importing Libs

In [ ]:
import os
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.estimator import Estimator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow_estimator.python.estimator.mode_keys import ModeKeys

# Importing the dataset

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data = pd.read_csv(os.path.join(dirname, filename))
data.head()

# Data info

In [ ]:
data.describe().T.style.bar()

In [ ]:
data.info()

In [ ]:
print('We have', data.shape[0], 'Rows and', data.shape[1], 'features')

In [ ]:
df = data.copy()
df.columns

In [ ]:
y = df['ProdTaken']
df = df.drop(['ProdTaken', 'CustomerID', 'TypeofContact'], axis = 1)

# Check the missing values

In [ ]:
df.isnull().sum(axis=0)

<h5> To deal with the <span style='background:yellow'> missing values </span> as we have to different feature types, I considered two different scenarios, the average for numerical missed values and high frequency used values for the categorical ones. </h5>


In [ ]:
df.fillna(df.mean(), inplace=True)

In [ ]:
categorical_cl = [ 'Occupation',  'Gender',
                  'ProductPitched', 'MaritalStatus',  'Designation']


for i in categorical_cl:
    le = LabelEncoder()
    n = str(i)+'_n'
    df[n] = le.fit_transform(df[i])
    del df[i]

# Normalizing the input

In [ ]:
scale = StandardScaler()
scale.fit(df)
df = scale.transform(df)
df = pd.DataFrame(df)

# Building Model

## Feature engineering

Replace the header with String value 

In [ ]:
feature = []
for i in range(len(df.columns)):
  feature.append(str(i))

col_rename = {i:j for i,j in zip(df.columns, feature)}
df = df.rename(columns=col_rename, inplace=False)
df.head()

In [ ]:
X = df

In [ ]:
dftrain, dfeval, y_train, y_eval = train_test_split(
    X, y, test_size=0.3, random_state=2)

Changing datatype to be compatible with the model

In [ ]:
dftrain = dftrain.astype("int64")
y_train = y_train.astype("int64")
dfeval = dfeval.astype("int64")
y_eval = y_eval.astype("int64")

In [ ]:
NUM_EXAMPLES = len(y_train)


def make_input_fn(X, y, n_epochs=None, shuffle=True):
    def input_fn():
        dataset = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
            dataset = dataset.shuffle(NUM_EXAMPLES)

        dataset = dataset.repeat(n_epochs)

        dataset = dataset.batch(NUM_EXAMPLES)
        return dataset
    return input_fn

# Training input functions

In [ ]:
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)


NUMERIC_COLUMNS = featuer


feature_columns = []


for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,
                                                            dtype=tf.float32))


est = tf.estimator.BoostedTreesClassifier(feature_columns,
                                          n_batches_per_layer=1,
                                          n_classes=2,
                                          n_trees=100,
                                          max_depth=10,
                                          learning_rate=0.1,
                                          center_bias=True)

est.train(train_input_fn, max_steps=100)


# Model Evaluation

In [ ]:
history = est.evaluate(eval_input_fn)

In [ ]:
items = list(history.items())
array = np.array(items)
print('Accuracy:', '{0:.0f}%'.format(array[0, 1].astype('float64') * 100))

# DFCs

In [ ]:
pred_dicts = list(est.experimental_predict_with_explanations(eval_input_fn))

In [ ]:
labels = y_eval.values
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])
df_dfc = pd.DataFrame([pred['dfc'] for pred in pred_dicts])
df_dfc.describe().T